# Regresión Polinómica vs Splines vs Natural Splines en Python

En este notebook replicamos el análisis del script R que compara diferentes técnicas de regresión no lineal.

**Objetivo**: Comparar el desempeño de diferentes enfoques de regresión no paramétrica:
- Regresión polinómica
- Regresión lineal local (por segmentos)
- Regresión por partes (Piecewise/Broken Stick Regression)
- Splines cúbicas (B-splines)
- Natural Splines

**Dataset**: `mcycle` de la librería MASS de R - datos de aceleración de cabeza en accidentes de motocicleta.

Referencia: Ripley (2002): "Modern Applied Statistics with S-PLUS". Springer.

## 1. Importación de Librerías

In [ ]:
# Manejo de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score

# Splines
from scipy.interpolate import BSpline, splrep, splev
from patsy import dmatrix, build_design_matrices

# Configuración de gráficos
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

# Desactivar warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Carga del Dataset `mcycle`

El dataset **mcycle** contiene mediciones de aceleración de la cabeza en una simulación de accidente de motocicleta.

**Variables**:
- `times`: Tiempo después del impacto (milisegundos)
- `accel`: Aceleración de la cabeza (unidades de gravedad, g)

El dataset contiene 133 observaciones y es comúnmente usado para ilustrar técnicas de regresión no lineal debido a su relación compleja y no monotónica.

In [ ]:
# Carga del dataset desde URL pública
url = "https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/MASS/mcycle.csv"
mcycle = pd.read_csv(url)

# Eliminar columna de índice si existe
if 'Unnamed: 0' in mcycle.columns:
    mcycle = mcycle.drop('Unnamed: 0', axis=1)

# Renombrar columnas si es necesario para consistencia
if 'rownames' in mcycle.columns:
    mcycle = mcycle.drop('rownames', axis=1)

print("Primeras 5 filas del dataset:")
print(mcycle.head())
print(f"\nDimensiones del dataset: {mcycle.shape}")
print(f"\nEstadísticas descriptivas:")
print(mcycle.describe())

## 3. Visualización Inicial de los Datos

In [ ]:
# Extraer variables
times = mcycle['times'].values
accel = mcycle['accel'].values

# Gráfico de dispersión
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.7, s=50, edgecolors='none')
plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title('Dataset mcycle: Aceleración vs Tiempo', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Regresión Polinómica (Grado 9)

La **regresión polinómica** extiende la regresión lineal al agregar términos de potencias superiores de las variables predictoras.

**Modelo**: $y = \beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_9 x^9 + \epsilon$

**Ventajas**:
- Fácil de implementar
- Modelo global (una función para todo el dominio)

**Desventajas**:
- Puede generar sobreajuste
- Comportamiento errático en los extremos
- No es local (un cambio en los datos afecta toda la curva)

La función `PolynomialFeatures` construye la matriz de diseño con términos polinómicos.

In [ ]:
# Crear características polinómicas de grado 9
poly_features = PolynomialFeatures(degree=9, include_bias=False)
X_poly = poly_features.fit_transform(times.reshape(-1, 1))

print("Matriz de diseño polinómica (primeras 5 filas):")
print(pd.DataFrame(X_poly[:5], columns=[f'times^{i+1}' for i in range(9)]))
print(f"\nDimensiones de la matriz de diseño: {X_poly.shape}")

In [ ]:
# Ajustar modelo de regresión polinómica
poly9_model = LinearRegression()
poly9_model.fit(X_poly, accel)

# Predicciones
poly9_pred = poly9_model.predict(X_poly)

# Resumen del modelo
r2_poly = r2_score(accel, poly9_pred)
mse_poly = mean_squared_error(accel, poly9_pred)

print("=" * 60)
print("RESUMEN DEL MODELO - REGRESIÓN POLINÓMICA (GRADO 9)")
print("=" * 60)
print(f"R² (coeficiente de determinación): {r2_poly:.4f}")
print(f"MSE (error cuadrático medio): {mse_poly:.4f}")
print(f"\nCoeficientes del modelo:")
for i, coef in enumerate(poly9_model.coef_):
    print(f"  β_{i+1} (times^{i+1}): {coef:.6e}")
print(f"  β_0 (intercepto): {poly9_model.intercept_:.6f}")
print("=" * 60)

In [ ]:
# Visualización de la regresión polinómica
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.7, s=50, label='Datos observados')
plt.plot(times, poly9_pred, color='blue', alpha=0.6, linewidth=3, 
         linestyle='--', label='Regresión Polinómica (grado 9)')
plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title('Regresión Polinómica (Grado 9)', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Regresión Lineal Local (por segmentos)

Una alternativa simple es ajustar **modelos lineales independientes** en diferentes regiones del dominio.

**Enfoque**: Dividir el rango de `times` en 5 segmentos y ajustar una regresión lineal en cada uno:
1. times ≤ 15
2. 15 < times ≤ 22
3. 22 < times ≤ 35
4. 35 < times ≤ 50
5. times > 50

**Problema**: Las funciones resultantes no son continuas en los puntos de corte ("knots").

In [ ]:
# Definir los segmentos
segments = [
    (times <= 15, 'times ≤ 15'),
    ((times > 15) & (times <= 22), '15 < times ≤ 22'),
    ((times > 22) & (times <= 35), '22 < times ≤ 35'),
    ((times > 35) & (times <= 50), '35 < times ≤ 50'),
    (times > 50, 'times > 50')
]

# Ajustar modelos lineales para cada segmento
local_models = []
local_preds = []

for mask, label in segments:
    if np.sum(mask) > 0:  # Verificar que hay datos en el segmento
        X_seg = times[mask].reshape(-1, 1)
        y_seg = accel[mask]
        
        model = LinearRegression()
        model.fit(X_seg, y_seg)
        pred = model.predict(X_seg)
        
        local_models.append((mask, model, label))
        local_preds.append((X_seg.flatten(), pred))
        
        print(f"{label}: β_0 = {model.intercept_:.3f}, β_1 = {model.coef_[0]:.3f}")

In [ ]:
# Visualización de regresión lineal local
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.7, s=50, label='Datos observados')

# Graficar cada segmento
for X_seg, pred in local_preds:
    plt.plot(X_seg, pred, color='red', alpha=0.6, linewidth=3, linestyle='--')

# Crear una sola etiqueta para la leyenda
plt.plot([], [], color='red', alpha=0.6, linewidth=3, 
         linestyle='--', label='Regresión lineal local (5 segmentos)')

plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title('Regresión Lineal Local por Segmentos', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Regresión por Partes (Piecewise / Broken Stick Regression)

La **regresión por partes** mejora la regresión lineal local al garantizar **continuidad** en los knots.

**Modelo aditivo**: Se usa una base de funciones tipo "bisagra" (hinge functions):

$$f(x) = \beta_0 + \beta_1 x + \beta_2 (x - \nu_1)_+ + \beta_3 (x - \nu_2)_+ + ... + \beta_k (x - \nu_k)_+$$

donde $(x - \nu)_+ = \max(0, x - \nu)$ es la función "parte positiva".

**Knots utilizados**: $\nu = \{15, 22, 35, 50\}$

**Ventaja**: La función resultante es continua pero no diferenciable en los knots.

In [ ]:
# Definir los knots
knots = [15, 22, 35, 50]

# Crear matriz de diseño para regresión por partes
def create_piecewise_features(x, knots):
    """Crea matriz de diseño con funciones hinge (x - knot)_+"""
    X = np.column_stack([x] + [np.maximum(0, x - k) for k in knots])
    return X

X_piecewise = create_piecewise_features(times, knots)

print("Matriz de diseño para regresión por partes (primeras 5 filas):")
feature_names = ['times'] + [f'(times-{k})_+' for k in knots]
print(pd.DataFrame(X_piecewise[:5], columns=feature_names))
print(f"\nDimensiones: {X_piecewise.shape}")

In [ ]:
# Ajustar modelo de regresión por partes
piecewise_model = LinearRegression()
piecewise_model.fit(X_piecewise, accel)

# Predicciones
piecewise_pred = piecewise_model.predict(X_piecewise)

# Resumen del modelo
r2_piecewise = r2_score(accel, piecewise_pred)
mse_piecewise = mean_squared_error(accel, piecewise_pred)

print("=" * 60)
print("RESUMEN DEL MODELO - REGRESIÓN POR PARTES")
print("=" * 60)
print(f"R² (coeficiente de determinación): {r2_piecewise:.4f}")
print(f"MSE (error cuadrático medio): {mse_piecewise:.4f}")
print(f"\nCoeficientes del modelo:")
print(f"  β_0 (intercepto): {piecewise_model.intercept_:.4f}")
for i, (name, coef) in enumerate(zip(feature_names, piecewise_model.coef_)):
    print(f"  β_{i+1} ({name}): {coef:.4f}")
print("=" * 60)

In [ ]:
# Visualización de regresión por partes
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.7, s=50, label='Datos observados')
plt.plot(times, piecewise_pred, color='green', alpha=0.6, linewidth=3, 
         linestyle='--', label='Regresión por partes')
plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title('Regresión por Partes (Broken Stick)', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Splines Cúbicas (B-splines de grado 3)

Los **splines cúbicos** son funciones polinómicas por partes que garantizan continuidad y diferenciabilidad.

**Propiedades**:
- Polinomios de grado 3 entre cada par de knots
- Continuidad de la función: $f(x)$
- Continuidad de la primera derivada: $f'(x)$
- Continuidad de la segunda derivada: $f''(x)$

**B-splines**: Bases de splines que forman una base computacionalmente estable.

La función `bs()` de `patsy` construye la matriz de diseño con bases B-spline, similar a la función `bs()` de R.

In [ ]:
# Crear matriz de diseño con B-splines usando patsy
# La sintaxis bs(x, knots=..., degree=3) es equivalente a la de R
spline_basis = dmatrix("bs(times, knots=knots, degree=3, include_intercept=False)", 
                       {"times": times, "knots": knots}, 
                       return_type='dataframe')

print("Matriz de diseño con B-splines (primeras 5 filas):")
print(spline_basis.head())
print(f"\nDimensiones: {spline_basis.shape}")
print(f"\nNúmero de bases: {spline_basis.shape[1]}")

In [ ]:
# Ajustar modelo de splines cúbicas
spline_model = LinearRegression()
spline_model.fit(spline_basis, accel)

# Predicciones
spline_pred = spline_model.predict(spline_basis)

# Resumen del modelo
r2_spline = r2_score(accel, spline_pred)
mse_spline = mean_squared_error(accel, spline_pred)

print("=" * 60)
print("RESUMEN DEL MODELO - SPLINES CÚBICAS")
print("=" * 60)
print(f"R² (coeficiente de determinación): {r2_spline:.4f}")
print(f"MSE (error cuadrático medio): {mse_spline:.4f}")
print(f"\nCoeficientes del modelo:")
print(f"  β_0 (intercepto): {spline_model.intercept_:.4f}")
for i, coef in enumerate(spline_model.coef_):
    print(f"  β_{i+1}: {coef:.4f}")
print("=" * 60)

In [ ]:
# Visualización de splines cúbicas
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.7, s=50, label='Datos observados')
plt.plot(times, spline_pred, color='brown', alpha=0.8, linewidth=3, 
         linestyle='--', label='Splines cúbicas (B-splines)')
plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title('Regresión con Splines Cúbicas (degree=3)', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Natural Splines

Los **natural splines** son splines cúbicos con restricciones adicionales en los extremos.

**Restricción**: En las regiones más allá de los knots extremos (bordes), la función es lineal.
- Para $x < \nu_1$ (primer knot): $f''(x) = 0$ → función lineal
- Para $x > \nu_k$ (último knot): $f''(x) = 0$ → función lineal

**Ventaja**: Reduce el sobreajuste en los extremos del dominio, donde típicamente hay menos datos.

La función `cr()` de `patsy` crea bases de natural splines (cubic regression splines), equivalente a `ns()` en R.

In [ ]:
# Crear matriz de diseño con Natural Splines usando patsy
# cr() en patsy es equivalente a ns() en R
nat_spline_basis = dmatrix("cr(times, knots=knots)", 
                           {"times": times, "knots": knots}, 
                           return_type='dataframe')

print("Matriz de diseño con Natural Splines (primeras 5 filas):")
print(nat_spline_basis.head())
print(f"\nDimensiones: {nat_spline_basis.shape}")
print(f"\nNúmero de bases: {nat_spline_basis.shape[1]}")

In [ ]:
# Ajustar modelo de natural splines
nat_spline_model = LinearRegression()
nat_spline_model.fit(nat_spline_basis, accel)

# Predicciones
nat_spline_pred = nat_spline_model.predict(nat_spline_basis)

# Resumen del modelo
r2_nat = r2_score(accel, nat_spline_pred)
mse_nat = mean_squared_error(accel, nat_spline_pred)

print("=" * 60)
print("RESUMEN DEL MODELO - NATURAL SPLINES")
print("=" * 60)
print(f"R² (coeficiente de determinación): {r2_nat:.4f}")
print(f"MSE (error cuadrático medio): {mse_nat:.4f}")
print(f"\nCoeficientes del modelo:")
print(f"  β_0 (intercepto): {nat_spline_model.intercept_:.4f}")
for i, coef in enumerate(nat_spline_model.coef_):
    print(f"  β_{i+1}: {coef:.4f}")
print("=" * 60)

In [ ]:
# Visualización de natural splines
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.7, s=50, label='Datos observados')
plt.plot(times, nat_spline_pred, color='orange', alpha=0.8, linewidth=3, 
         linestyle='--', label='Natural Splines')
plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title('Regresión con Natural Splines', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nNota: Observa cómo la función se comporta de forma lineal en los bordes.")

## 9. Visualización Comparativa (estilo ggplot2)

Comparación visual de los tres métodos principales:
- Regresión polinómica (grado 9) - azul
- Splines cúbicas - marrón
- Natural splines - naranja

In [ ]:
# Gráfico comparativo estilo ggplot2 con theme_minimal
plt.figure(figsize=(12, 7))

# Datos observados
plt.scatter(times, accel, color='black', alpha=0.55, s=50, 
           label='Datos observados', zorder=5)

# Ordenar los datos para graficar líneas suaves
sort_idx = np.argsort(times)
times_sorted = times[sort_idx]

# Regresión polinómica
plt.plot(times_sorted, poly9_pred[sort_idx], color='blue', 
         linewidth=2.5, label='Regresión Polinómica (grado 9)', alpha=0.8)

# Splines cúbicas
plt.plot(times_sorted, spline_pred[sort_idx], color='brown', 
         linewidth=2.5, label='Splines Cúbicas', alpha=0.8)

# Natural splines
plt.plot(times_sorted, nat_spline_pred[sort_idx], color='orange', 
         linewidth=2.5, label='Natural Splines', alpha=0.8)

plt.xlabel('Tiempo después del impacto (ms)', fontsize=13)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=13)
plt.title('Comparación de Métodos de Regresión No Lineal', 
         fontsize=15, fontweight='bold', pad=15)
plt.legend(fontsize=11, loc='best', framealpha=0.9)
plt.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

### Tabla Comparativa de Métricas

In [ ]:
# Crear tabla comparativa de métricas
comparison_df = pd.DataFrame({
    'Método': ['Regresión Polinómica (grado 9)', 'Regresión por Partes', 
               'Splines Cúbicas', 'Natural Splines'],
    'R²': [r2_poly, r2_piecewise, r2_spline, r2_nat],
    'MSE': [mse_poly, mse_piecewise, mse_spline, mse_nat]
})

print("\n" + "=" * 70)
print("COMPARACIÓN DE MÉTRICAS DE LOS MODELOS")
print("=" * 70)
print(comparison_df.to_string(index=False))
print("=" * 70)
print("\nObservaciones:")
print("- R² más alto indica mejor ajuste a los datos de entrenamiento")
print("- MSE más bajo indica menores errores de predicción")
print("- Sin embargo, R² alto puede indicar sobreajuste si no se valida")

## 10. Leave-One-Out Cross-Validation (LOOCV)

Para seleccionar el **número óptimo de grados de libertad (df)** en splines, usamos validación cruzada.

**Leave-One-Out CV**:
1. Para cada observación $i$:
   - Entrenar el modelo con todas las observaciones excepto $i$
   - Predecir el valor de $i$
   - Calcular el error cuadrático
2. Promediar los errores sobre todas las observaciones

**Ventaja**: Usa todos los datos para entrenamiento (excepto uno), maximizando el uso de información.

**Desventaja**: Computacionalmente costoso (N iteraciones).

Probaremos modelos con df (degrees of freedom) de 3 a 15 y seleccionaremos el que minimice el error LOOCV.

In [ ]:
# LOOCV para seleccionar grados de libertad óptimos
df_range = range(3, 16)  # df de 3 a 15
cv_errors = []

print("Ejecutando Leave-One-Out Cross-Validation...")
print("df\tMSE (LOOCV)")
print("-" * 30)

for df in df_range:
    # Crear modelo con df grados de libertad
    # df en bs() controla el número de bases (knots se colocan automáticamente)
    spline_basis_df = dmatrix(f"bs(times, df={df}, degree=3, include_intercept=False)", 
                               {"times": times}, 
                               return_type='dataframe')
    
    # LOOCV
    loo = LeaveOneOut()
    mse_scores = []
    
    for train_idx, test_idx in loo.split(spline_basis_df):
        # Split train/test
        X_train, X_test = spline_basis_df.iloc[train_idx], spline_basis_df.iloc[test_idx]
        y_train, y_test = accel[train_idx], accel[test_idx]
        
        # Entrenar y predecir
        model_cv = LinearRegression()
        model_cv.fit(X_train, y_train)
        y_pred = model_cv.predict(X_test)
        
        # Error cuadrático
        mse_scores.append((y_test[0] - y_pred[0])**2)
    
    # MSE promedio
    cv_mse = np.mean(mse_scores)
    cv_errors.append(cv_mse)
    print(f"{df}\t{cv_mse:.2f}")

# Encontrar df óptimo
optimal_df = df_range[np.argmin(cv_errors)]
min_error = np.min(cv_errors)

print("-" * 30)
print(f"\nGrados de libertad óptimos: {optimal_df}")
print(f"MSE mínimo (LOOCV): {min_error:.2f}")

In [ ]:
# Visualización de errores LOOCV
plt.figure(figsize=(10, 6))
plt.plot(df_range, cv_errors, 'o-', linewidth=2.5, markersize=8, color='darkblue')
plt.axvline(x=optimal_df, color='red', linestyle='--', linewidth=2, 
           label=f'df óptimo = {optimal_df}')
plt.xlabel('Grados de Libertad (df)', fontsize=12)
plt.ylabel('MSE (Leave-One-Out CV)', fontsize=12)
plt.title('Selección de Grados de Libertad mediante LOOCV', 
         fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Ajuste Final con df Óptimo

In [ ]:
# Ajustar modelo final con df óptimo
spline_basis_optimal = dmatrix(f"bs(times, df={optimal_df}, degree=3, include_intercept=False)", 
                               {"times": times}, 
                               return_type='dataframe')

spline_optimal_model = LinearRegression()
spline_optimal_model.fit(spline_basis_optimal, accel)
spline_optimal_pred = spline_optimal_model.predict(spline_basis_optimal)

r2_optimal = r2_score(accel, spline_optimal_pred)
mse_optimal = mean_squared_error(accel, spline_optimal_pred)

print("=" * 60)
print(f"MODELO ÓPTIMO - SPLINES CON df={optimal_df}")
print("=" * 60)
print(f"R² (coeficiente de determinación): {r2_optimal:.4f}")
print(f"MSE (error en conjunto de entrenamiento): {mse_optimal:.4f}")
print(f"MSE (estimado por LOOCV): {min_error:.4f}")
print("=" * 60)

In [ ]:
# Visualización del modelo óptimo
plt.figure(figsize=(10, 6))
plt.scatter(times, accel, color='black', alpha=0.6, s=50, label='Datos observados')
plt.plot(times_sorted, spline_optimal_pred[sort_idx], color='darkgreen', 
         linewidth=3, label=f'Spline óptima (df={optimal_df})', alpha=0.9)
plt.xlabel('Tiempo después del impacto (ms)', fontsize=12)
plt.ylabel('Aceleración de la cabeza (g)', fontsize=12)
plt.title(f'Modelo Óptimo: Spline Cúbica con df={optimal_df}', 
         fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Conclusiones

En este notebook hemos replicado el análisis del script R comparando diferentes técnicas de regresión no lineal:

### Resumen de Métodos:

1. **Regresión Polinómica**: Simple pero propensa a sobreajuste y comportamiento errático en los extremos.

2. **Regresión Lineal Local**: Fácil de interpretar pero discontinua en los knots.

3. **Regresión por Partes**: Garantiza continuidad pero no diferenciabilidad.

4. **Splines Cúbicas**: Continuas y dos veces diferenciables, excelente balance entre flexibilidad y suavidad.

5. **Natural Splines**: Como splines cúbicas pero con restricciones lineales en los bordes para reducir sobreajuste.

### Ventajas de Splines:
- Flexibilidad local: cambios en una región no afectan regiones lejanas
- Continuidad y diferenciabilidad
- Comportamiento estable en los extremos (natural splines)
- Se pueden optimizar vía cross-validación

### Cross-Validación:
Leave-One-Out CV permite seleccionar el número óptimo de grados de libertad, balanceando:
- **Sesgo**: df muy bajo → modelo demasiado simple (underfitting)
- **Varianza**: df muy alto → modelo demasiado complejo (overfitting)

---

**Referencias**:
- Ripley, B. D. (2002). Modern Applied Statistics with S-PLUS. Springer.
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). The Elements of Statistical Learning.